In [37]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time


In [2]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

In [3]:
def get_links(cateId, page):
    
    # 한국소설 25, 일본소설38, 영미소설 51, 기타외국 63
    url ='http://www.bandinlunis.com/front/product/bookCategoryMain.do?cateId={}'.format(cateId)

    params = {
        'listSize':60,
        'viewType':'Detail',
        'listType':'all',
        'sort':'sort1',
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('dl.prod_info > dt > a')

    return tuple(each['href'] for each in aList)

In [4]:
ko_links = [] # 한국 소설 url
jp_links = [] # 일본 소설 url
en_links = [] # 영미 소설 url
etc_links = [] # 기타 소설 url

for page in tqdm(range(1,6)) :
    ko_links.extend(get_links(25,page))

for page in tqdm(range(1,6)) :
    jp_links.extend(get_links(38,page))
    
for page in tqdm(range(1,6)) :
    en_links.extend(get_links(51,page))
    
for page in tqdm(range(1,6)) :
    etc_links.extend(get_links(63,page))

100%|██████████| 5/5 [00:07<00:00,  1.53s/it]


In [5]:
ko_links[0]

'/front/product/detailProduct.do?prodId=4002576'

In [63]:
def get_info(page_url):

    url = 'http://www.bandinlunis.com' + page_url

    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    try :
        name = soup.select_one('div.group_title > h1 > span').text # 책 제목
    except Exception as e :
        name = ""
        print(e)
    
    try :
        author = soup.select_one('div.group_inside > ul a').text # 작가 이름
    except Exception as e :
        author = ""
        print(e)    

    try :
        text = ""
        text += soup.select_one('#cutBookDesc').getText() # 텍스트
    except Exception as e :
        text = ""
        print(e)
    
    try :
        image = soup.select_one('#wrapDetail > div.wrap_contents > div.inner_left > div.thumb_bookCover > img')['src']
    except Exception as e :
        image = ""
        print(e)
    
    return (name, author, text, image)

In [64]:
get_info(ko_links[0])

('82년생 김지영',
 '조남주',
 "\n출판사 리뷰\n“사람들이 나보고 맘충이래.”한국에서 여자로 살아가는 일그 공포, 피로, 당황, 놀람, 혼란, 좌절의연속에 대한 인생 현장 보고서 조남주 장편소설 『82년생 김지영』이 민음사 ‘오늘의 젊은 작가’ 시리즈로 출간되었다. 조남주 작가는 2011년, 지적 장애가 있는 한 소년의 재능이 발견되면서 벌어지는 사건을 통해 삶의 부조리를 현실적이면서도 따뜻하게 그려낸 작품 『귀를 귀울이면』으로 ‘문학동네소설상’을 받으며 데뷔했다. 시사 교양 프로그램에서 10년 동안 일한 방송 작가답게 서민들의 일상에서 발생하는 비극을 사실적이고 공감대 높은 스토리로 표현하는 데 특출 난 재능을 보이는 작가는 신작 『82년생 김지영』에서 30대를 살고 있는 한국 여성들의 보편적인 일상을 완벽하게 재현한다.주인공 ‘김지영 씨’의 기억을 바탕으로 한 고백을 한 축으로, 고백을 뒷받침하는 각종 통계 자료와 기사들을 또 다른 축으로 삼는 이 소설은 1982년생 김지영 씨로 대변되는 ‘그녀’들의 인생 마디마디에 존재하는 성차별적 요소를 핍진하게 묘사한다. 이를 통해 작가는 제도적 성차별이 줄어든 시대의 보이지 않는 차별들이 어떻게 여성들의 삶을 제약하고 억압하는지 보여 준다. 여권이 신장된 시대, 그러나 여전히 ‘여성’이라는 조건이 굴레로 존재하는 사회에서 살아가는 한 여자의 인생을 다룬 『82년생 김지영』은 조용한 고백과 뜨거운 고발로 완성된 새로운 페미니즘 소설이자 수많은 사람들의 경험과 자료로 이루어진 ‘목소리 소설’이다. 맘충이, 여혐, 메갈리아 등 연일 새롭게 등장하는 페미니즘 화두를 관심 있게 지켜보는 독자라면 누구나 흥미롭게 읽을 수 있고 저마다 의미를 발견할 수 있을 것이다. ■“사람들이 나보고 맘충이래.” 엄마를 뜻하는 ‘맘(Mom)’과 벌레를 뜻하는 ‘충(蟲)’의 합성어인 ‘맘충’은 제 아이만 싸고도는 일부 몰상식한 엄마를 가리키는 용어다. 그러나 ‘맘충’이란 호칭은 육아하는 엄마 대부분에게 무차별적으로 사용되며 많은 여성

In [65]:
ko_info = []
for each_link in tqdm(ko_links) :
    ko_info.append(get_info(each_link))

 24%|██▎       | 71/300 [01:09<03:44,  1.02it/s]

'NoneType' object has no attribute 'getText'


 51%|█████     | 152/300 [02:29<02:25,  1.01it/s]

'NoneType' object has no attribute 'getText'


 56%|█████▌    | 167/300 [02:44<02:11,  1.01it/s]

...

 70%|███████   | 211/300 [03:31<01:29,  1.00s/it]

'NoneType' object has no attribute 'getText'


 71%|███████▏  | 214/300 [03:34<01:26,  1.00s/it]

'NoneType' object has no attribute 'getText'


100%|██████████| 300/300 [05:15<00:00,  1.05s/it]


In [69]:
ko_data = pd.DataFrame(ko_info,columns=['name','author','text','image'])
ko_data.sample(20)

,name,author,text,image
186,현의 노래-개정판,김훈,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t역사적 상상력을 극한까...,http://image.bandinlunis.com/upload/product/la...
104,우아한 거짓말,김려령,\n출판사 리뷰\n믿고 보는 원작의 힘 영화 「완득이」 제작진과 김려령 작가가 다시...,http://image.bandinlunis.com/upload/product/37...
84,연어 - 100만 부 기념 특별 한정판 한글 + 영문,안도현,"\n이 책은\n20만 부 이상 팔린 베스트 셀러. 어른들을 위한 동화라는, 국내에서...",http://image.bandinlunis.com/upload/product/34...
192,이별이 떠났다,소재원,"\n이 책은\n“엄마도 울고, 딸도 울었다.”\n \n드라마와 또 다른 원작이 주는...",http://image.bandinlunis.com/upload/product/41...
25,도가니,공지영,"\n출판사 리뷰\n작가 공지영, 거짓과 맞서 희망을 쓰다! “진실을 결코 개들에게 ...",http://image.bandinlunis.com/upload/product/38...
17,미스 함무라비,문유석,\n출판사 리뷰\n현직 부장판사 문유석이 써내려간 법정 활극!\n“권리 위에 잠자는...,http://image.bandinlunis.com/upload/product/40...
226,골든 슬럼버 [특별판],이사카 고타로,"\n이 책은\n영화 <골든 슬럼버> (주연 강동원, 감독 노동석) 원작 소설!제5회...",http://image.bandinlunis.com/upload/product/41...
110,칼의 노래 (양장),김훈,"\n출판사 리뷰\n1993년 12월, 한국문학의 새로운 플랫폼이고자 문을 열었던 문...",http://image.bandinlunis.com/upload/product/37...
50,개밥바라기 별,황석영,"\n이 책은\n우리 한때, 아프고 힘들지 않았다면 어떻게 지난날을 ‘축제’였다고 말...",http://image.bandinlunis.com/upload/product/28...
115,강남몽,황석영,\n출판사 리뷰\n“우리의 뜨겁고 슬픈 꿈은 어디로 갔을까?“ \r\n작가 황석영이...,http://image.bandinlunis.com/upload/product/31...


In [74]:
jp_info = []
for each_link in tqdm(jp_links) :
    try:
        jp_info.append(get_info(each_link))
    except Exception as e :
        print(e)
        continue



  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 1/300 [00:00<04:45,  1.05it/s]

  1%|          | 2/300 [00:01<04:48,  1.03it/s]

  1%|          | 3/300 [00:02<04:08,  1.20it/s]

  1%|▏         | 4/300 [00:03<03:53,  1.27it/s]

  2%|▏         | 5/300 [00:04<03:56,  1.25it/s]

  2%|▏         | 6/300 [00:04<03:42,  1.32it/s]

  2%|▏         | 7/300 [00:05<03:44,  1.30it/s]

  3%|▎         | 8/300 [00:06<03:55,  1.24it/s]

  3%|▎         | 9/300 [00:07<03:47,  1.28it/s]

  3%|▎         | 10/300 [00:08<03:54,  1.24it/s]

  4%|▎         | 11/300 [00:08<03:50,  1.25it/s]

  4%|▍         | 12/300 [00:09<03:53,  1.24it/s]

  4%|▍         | 13/300 [00:10<03:51,  1.24it/s]

  5%|▍         | 14/300 [00:11<03:45,  1.27it/s]

  5%|▌         | 15/300 [00:11<03:47,  1.25it/s]

  5%|▌         | 16/300 [00:13<03:50,  1.23it/s]

  6%|▌         | 17/300 [00:14<03:59,  1.18it/s]

  6%|▌         | 18/300 [00:15<03:56,  1.19it/s]

  6%|▋         | 19/300 [00:16<03:59,  1.17it/s]

  7%|▋         |

 48%|████▊     | 145/300 [02:16<02:25,  1.07it/s]

 49%|████▊     | 146/300 [02:17<02:24,  1.06it/s]

 49%|████▉     | 147/300 [02:18<02:23,  1.06it/s]

 49%|████▉     | 148/300 [02:19<02:22,  1.06it/s]

 50%|████▉     | 149/300 [02:19<02:21,  1.07it/s]

 50%|█████     | 150/300 [02:20<02:20,  1.06it/s]

 50%|█████     | 151/300 [02:21<02:19,  1.06it/s]

 51%|█████     | 152/300 [02:22<02:19,  1.06it/s]

 51%|█████     | 153/300 [02:23<02:18,  1.06it/s]

 51%|█████▏    | 154/300 [02:24<02:16,  1.07it/s]

 52%|█████▏    | 155/300 [02:24<02:15,  1.07it/s]

 52%|█████▏    | 156/300 [02:25<02:14,  1.07it/s]

 52%|█████▏    | 157/300 [02:25<02:12,  1.08it/s]

 53%|█████▎    | 158/300 [02:26<02:11,  1.08it/s]

 53%|█████▎    | 159/300 [02:27<02:11,  1.08it/s]

 53%|█████▎    | 160/300 [02:29<02:10,  1.07it/s]

 54%|█████▎    | 161/300 [02:29<02:09,  1.07it/s]

 54%|█████▍    | 162/300 [02:30<02:08,  1.07it/s]

 54%|█████▍    | 163/300 [02:32<02:08,  1.07it/s]

 55%|█████▍    | 164/300 [02:33

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'getText'
'NoneType' object is not subscriptable
local variable 'name' referenced before assignment




 66%|██████▌   | 197/300 [03:01<01:35,  1.08it/s]

 66%|██████▌   | 198/300 [03:02<01:34,  1.08it/s]

 66%|██████▋   | 199/300 [03:03<01:33,  1.08it/s]

 67%|██████▋   | 200/300 [03:04<01:32,  1.08it/s]

 67%|██████▋   | 201/300 [03:05<01:31,  1.08it/s]

 67%|██████▋   | 202/300 [03:06<01:30,  1.08it/s]

 68%|██████▊   | 203/300 [03:07<01:29,  1.08it/s]

 68%|██████▊   | 204/300 [03:08<01:28,  1.08it/s]

 68%|██████▊   | 205/300 [03:09<01:27,  1.08it/s]

 69%|██████▊   | 206/300 [03:10<01:26,  1.08it/s]

 69%|██████▉   | 207/300 [03:11<01:25,  1.08it/s]

 69%|██████▉   | 208/300 [03:11<01:24,  1.08it/s]

 70%|██████▉   | 209/300 [03:12<01:23,  1.08it/s]

 70%|███████   | 210/300 [03:13<01:23,  1.08it/s]

 70%|███████   | 211/300 [03:14<01:22,  1.08it/s]

 71%|███████   | 212/300 [03:16<01:21,  1.08it/s]

 71%|███████   | 213/300 [03:16<01:20,  1.08it/s]

 71%|███████▏  | 214/300 [03:18<01:19,  1.08it/s]

 72%|███████▏  | 215/300 [03:19<01:18,  1.08it/s]

 72%|███████▏  | 216/300 [03:

'NoneType' object has no attribute 'getText'




 86%|████████▌ | 257/300 [03:59<00:40,  1.07it/s]

 86%|████████▌ | 258/300 [04:00<00:39,  1.07it/s]

 86%|████████▋ | 259/300 [04:02<00:38,  1.07it/s]

 87%|████████▋ | 260/300 [04:02<00:37,  1.07it/s]

 87%|████████▋ | 261/300 [04:03<00:36,  1.07it/s]

 87%|████████▋ | 262/300 [04:04<00:35,  1.07it/s]

 88%|████████▊ | 263/300 [04:04<00:34,  1.07it/s]

 88%|████████▊ | 264/300 [04:06<00:33,  1.07it/s]

 88%|████████▊ | 265/300 [04:07<00:32,  1.07it/s]

 89%|████████▊ | 266/300 [04:08<00:31,  1.07it/s]

 89%|████████▉ | 267/300 [04:09<00:30,  1.07it/s]

 89%|████████▉ | 268/300 [04:10<00:29,  1.07it/s]

 90%|████████▉ | 269/300 [04:11<00:28,  1.07it/s]

 90%|█████████ | 270/300 [04:12<00:28,  1.07it/s]

 90%|█████████ | 271/300 [04:13<00:27,  1.07it/s]

'NoneType' object has no attribute 'getText'




 91%|█████████ | 272/300 [04:14<00:26,  1.07it/s]

 91%|█████████ | 273/300 [04:15<00:25,  1.07it/s]

 91%|█████████▏| 274/300 [04:16<00:24,  1.07it/s]

 92%|█████████▏| 275/300 [04:17<00:23,  1.07it/s]

 92%|█████████▏| 276/300 [04:18<00:22,  1.07it/s]

 92%|█████████▏| 277/300 [04:19<00:21,  1.07it/s]

 93%|█████████▎| 278/300 [04:20<00:20,  1.07it/s]

 93%|█████████▎| 279/300 [04:21<00:19,  1.07it/s]

 93%|█████████▎| 280/300 [04:22<00:18,  1.07it/s]

 94%|█████████▎| 281/300 [04:23<00:17,  1.07it/s]

 94%|█████████▍| 282/300 [04:24<00:16,  1.07it/s]

 94%|█████████▍| 283/300 [04:25<00:15,  1.07it/s]

 95%|█████████▍| 284/300 [04:26<00:15,  1.06it/s]

 95%|█████████▌| 285/300 [04:27<00:14,  1.06it/s]

 95%|█████████▌| 286/300 [04:28<00:13,  1.06it/s]

 96%|█████████▌| 287/300 [04:29<00:12,  1.06it/s]

 96%|█████████▌| 288/300 [04:30<00:11,  1.06it/s]

 96%|█████████▋| 289/300 [04:31<00:10,  1.06it/s]

 97%|█████████▋| 290/300 [04:32<00:09,  1.06it/s]

 97%|█████████▋| 291/300 [04:

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'getText'
'NoneType' object is not subscriptable
local variable 'name' referenced before assignment




100%|█████████▉| 299/300 [04:39<00:00,  1.07it/s]

100%|██████████| 300/300 [04:40<00:00,  1.07it/s]



In [77]:
jp_data = pd.DataFrame(jp_info,columns=['name','author','text','image'])
jp_data.sample(20)

,name,author,text,image
151,소문의 여자,오쿠다 히데오,\n출판사 리뷰\n오쿠다 히데오가 최초로 내놓은 범죄스릴러 열 개의 퍼즐을 맞추기 ...,http://image.bandinlunis.com/upload/product/36...
7,기사단장 죽이기 1,무라카미 하루키,"\n출판사 리뷰\n아내의 갑작스러운 이혼 통보 후,나는 산꼭대기 집에서 새로운 생활...",http://image.bandinlunis.com/upload/product/40...
193,걸어도 걸어도,고레에다 히로카즈,\n출판사 리뷰\n매 순간 모양과 빛깔을 달리하며 갱신되는 가족사진 같은 이야기 걷...,http://image.bandinlunis.com/upload/product/41...
252,술이 있으면 어디든 좋아,기타무라 가오루,\n이 책은\n술 하나로 도쿄 뒷골목을 점령한사랑스러운 괴짜 아가씨 코사카이의 기승...,http://image.bandinlunis.com/upload/product/39...
75,밀실 살인게임,우타노쇼고,\n출판사 리뷰\n▶추리소설의 기재(奇才) 우타노 쇼고 본격미스터리의 혼을 불사른 ...,http://image.bandinlunis.com/upload/product/32...
169,태엽감는 새 2 [개정],무라카미하루키,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t이 작품은 실은 바로 ...,http://image.bandinlunis.com/upload/product/40...
55,갈증,후카마치 아키오,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t일본 제3회 ‘이 미스...,http://image.bandinlunis.com/upload/product/41...
16,인간실격 - 세계문학전집 103(20쇄),다자이 오사무,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t작가가 자전적인 체험을...,http://image.bandinlunis.com/upload/product/29...
275,노르웨이의 숲 전 2권 상하세트,무라카미 하루키,\n이 책은\n우리나라에서 《상실의 시대》로 게재되어 널리 알려진 무라카미 하루키의...,http://image.bandinlunis.com/upload/product/la...
11,너의 췌장을 먹고 싶어,스미노 요루,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t- 일본 현지 80만 ...,http://image.bandinlunis.com/upload/product/40...


In [78]:
en_info = []
for each_link in tqdm(en_links) :
    try:
        en_info.append(get_info(each_link))
    except Exception as e :
        print(e)
        continue

en_data = pd.DataFrame(en_info,columns=['name','author','text','image'])
en_data.sample(20)



  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 1/300 [00:01<05:34,  1.12s/it]

  1%|          | 2/300 [00:01<04:25,  1.12it/s]

  1%|          | 3/300 [00:02<03:57,  1.25it/s]

  1%|▏         | 4/300 [00:03<04:19,  1.14it/s]

  2%|▏         | 5/300 [00:04<04:15,  1.15it/s]

  2%|▏         | 6/300 [00:05<04:22,  1.12it/s]

  2%|▏         | 7/300 [00:06<04:29,  1.09it/s]

  3%|▎         | 8/300 [00:07<04:29,  1.09it/s]

  3%|▎         | 9/300 [00:08<04:32,  1.07it/s]

  3%|▎         | 10/300 [00:09<04:40,  1.04it/s]

  4%|▎         | 11/300 [00:10<04:29,  1.07it/s]

  4%|▍         | 12/300 [00:11<04:26,  1.08it/s]

  4%|▍         | 13/300 [00:12<04:27,  1.07it/s]

  5%|▍         | 14/300 [00:12<04:19,  1.10it/s]

  5%|▌         | 15/300 [00:13<04:19,  1.10it/s]

  5%|▌         | 16/300 [00:14<04:22,  1.08it/s]

  6%|▌         | 17/300 [00:16<04:37,  1.02it/s]

  6%|▌         | 18/300 [00:17<04:36,  1.02it/s]

  6%|▋         | 19/300 [00:18<04:36,  1.02it/s]

  7%|▋         |

'NoneType' object has no attribute 'getText'




 32%|███▏      | 96/300 [01:37<03:27,  1.02s/it]

 32%|███▏      | 97/300 [01:38<03:26,  1.02s/it]

 33%|███▎      | 98/300 [01:39<03:25,  1.02s/it]

 33%|███▎      | 99/300 [01:40<03:24,  1.02s/it]

 33%|███▎      | 100/300 [01:41<03:23,  1.02s/it]

 34%|███▎      | 101/300 [01:42<03:21,  1.01s/it]

 34%|███▍      | 102/300 [01:43<03:21,  1.02s/it]

 34%|███▍      | 103/300 [01:44<03:20,  1.02s/it]

 35%|███▍      | 104/300 [01:45<03:19,  1.02s/it]

 35%|███▌      | 105/300 [01:46<03:18,  1.02s/it]

 35%|███▌      | 106/300 [01:47<03:17,  1.02s/it]

 36%|███▌      | 107/300 [01:48<03:15,  1.01s/it]

 36%|███▌      | 108/300 [01:49<03:14,  1.01s/it]

 36%|███▋      | 109/300 [01:50<03:14,  1.02s/it]

 37%|███▋      | 110/300 [01:51<03:13,  1.02s/it]

 37%|███▋      | 111/300 [01:53<03:12,  1.02s/it]

 37%|███▋      | 112/300 [01:54<03:11,  1.02s/it]

 38%|███▊      | 113/300 [01:55<03:10,  1.02s/it]

 38%|███▊      | 114/300 [01:56<03:09,  1.02s/it]

 38%|███▊      | 115/300 [01:56<0

'NoneType' object has no attribute 'getText'




 39%|███▉      | 118/300 [02:00<03:05,  1.02s/it]

...

'NoneType' object has no attribute 'getText'




 99%|█████████▉| 298/300 [04:56<00:01,  1.00it/s]

100%|█████████▉| 299/300 [04:57<00:00,  1.00it/s]

'NoneType' object has no attribute 'getText'




100%|██████████| 300/300 [04:58<00:00,  1.00it/s]



,name,author,text,image
52,햄릿[한글판+영문판],윌리엄 셰익스피어스,\n출판사 리뷰\n영국이 낳은 세계 최고의 시인 겸 극작가 셰익스피어 4대 비극의 ...,http://image.bandinlunis.com/upload/product/37...
58,[정가인하]오즈의 마법사[한글판+영문판],라이언 프랭크 바움,\n출판사 리뷰\n21세기 최고 할리우드 감독 샘 레이미가 선택한20세기 최고 환상...,http://image.bandinlunis.com/upload/product/37...
8,모든 것을 기억하는 남자,데이비드 발다치,\n출판사 리뷰\n★ 2015 아마존 베스트 오브 베스트셀러★ 뉴욕타임스 베스트셀러...,http://image.bandinlunis.com/upload/product/39...
43,템테이션 Temptation,더글라스 케네디,\n출판사 리뷰\n한 번의 성공이 반드시 ‘영원한 성공’으로 귀결되지는 않는다! ...,http://image.bandinlunis.com/upload/product/35...
231,행복한 그림자의 춤,앨리스 먼로,"\n출판사 리뷰\n\t\t\t\t\t\t\t\t\t\t\t\t\t♠""먼로는 하나의 ...",http://image.bandinlunis.com/upload/product/31...
177,노인과 바다 - 더 클래식 세계문학 미니미니북1,어니스트 헤밍웨이,\n출판사 리뷰\n\t\t\t\t\t\t\t\t\t\t\t\t\t도서출판 더클래식에...,http://image.bandinlunis.com/upload/product/41...
65,메이즈 러너,제임스 대시너,\n출판사 리뷰\n전 세계 스릴러 팬의 심장을 뛰게 만든 액션 블록버스터 ★아마존 ...,http://image.bandinlunis.com/upload/product/34...
28,비하인드 도어,B. A. 패리스,\n이 책은\n*《인터내셔널》《뉴욕타임스》 베스트셀러 *「링컨 차를 탄 변호사」제작...,http://image.bandinlunis.com/upload/product/40...
207,두 도시 이야기,찰스 디킨스,\n출판사 리뷰\n“「다크 나이트 라이즈」는 『두 도시 이야기』 그 자체이다.” ―...,http://image.bandinlunis.com/upload/product/35...
274,이런 사랑,이언매큐언,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t이런 사랑도 있을까? ...,http://image.bandinlunis.com/upload/product/28...


In [79]:
etc_info = []
for each_link in tqdm(etc_links) :
    try:
        etc_info.append(get_info(each_link))
    except Exception as e :
        print(e)
        continue

etc_data = pd.DataFrame(etc_info,columns=['name','author','text','image'])
etc_data.sample(20)



  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 1/300 [00:00<03:21,  1.48it/s]

  1%|          | 2/300 [00:01<03:54,  1.27it/s]

  1%|          | 3/300 [00:02<04:25,  1.12it/s]

  1%|▏         | 4/300 [00:03<04:05,  1.21it/s]

  2%|▏         | 5/300 [00:04<04:51,  1.01it/s]

  2%|▏         | 6/300 [00:06<04:56,  1.01s/it]

  2%|▏         | 7/300 [00:06<04:43,  1.03it/s]

  3%|▎         | 8/300 [00:07<04:34,  1.07it/s]

  3%|▎         | 9/300 [00:08<04:35,  1.06it/s]

  3%|▎         | 10/300 [00:09<04:26,  1.09it/s]

  4%|▎         | 11/300 [00:10<04:32,  1.06it/s]

  4%|▍         | 12/300 [00:11<04:33,  1.05it/s]

  4%|▍         | 13/300 [00:12<04:31,  1.06it/s]

  5%|▍         | 14/300 [00:13<04:43,  1.01it/s]

  5%|▌         | 15/300 [00:14<04:42,  1.01it/s]

  5%|▌         | 16/300 [00:16<04:45,  1.00s/it]

  6%|▌         | 17/300 [00:17<04:45,  1.01s/it]

  6%|▌         | 18/300 [00:18<04:42,  1.00s/it]

  6%|▋         | 19/300 [00:19<04:42,  1.01s/it]

  7%|▋         |

'NoneType' object has no attribute 'getText'




 37%|███▋      | 111/300 [01:56<03:18,  1.05s/it]

 37%|███▋      | 112/300 [01:57<03:16,  1.05s/it]

 38%|███▊      | 113/300 [01:58<03:15,  1.05s/it]

 38%|███▊      | 114/300 [01:58<03:13,  1.04s/it]

'NoneType' object has no attribute 'getText'




 38%|███▊      | 115/300 [01:59<03:12,  1.04s/it]

 39%|███▊      | 116/300 [02:00<03:11,  1.04s/it]

 39%|███▉      | 117/300 [02:02<03:10,  1.04s/it]

 39%|███▉      | 118/300 [02:02<03:09,  1.04s/it]

 40%|███▉      | 119/300 [02:04<03:08,  1.04s/it]

 40%|████      | 120/300 [02:04<03:07,  1.04s/it]

...

'NoneType' object has no attribute 'getText'




 93%|█████████▎| 278/300 [04:47<00:22,  1.03s/it]

 93%|█████████▎| 279/300 [04:48<00:21,  1.04s/it]

 93%|█████████▎| 280/300 [04:49<00:20,  1.04s/it]

 94%|█████████▎| 281/300 [04:50<00:19,  1.04s/it]

 94%|█████████▍| 282/300 [04:52<00:18,  1.04s/it]

 94%|█████████▍| 283/300 [04:53<00:17,  1.04s/it]

'NoneType' object has no attribute 'getText'




 95%|█████████▍| 284/300 [04:53<00:16,  1.04s/it]

 95%|█████████▌| 285/300 [04:54<00:15,  1.03s/it]

 95%|█████████▌| 286/300 [04:55<00:14,  1.03s/it]

 96%|█████████▌| 287/300 [04:56<00:13,  1.03s/it]

 96%|█████████▌| 288/300 [04:57<00:12,  1.03s/it]

 96%|█████████▋| 289/300 [04:58<00:11,  1.03s/it]

 97%|█████████▋| 290/300 [04:59<00:10,  1.03s/it]

 97%|█████████▋| 291/300 [05:00<00:09,  1.03s/it]

 97%|█████████▋| 292/300 [05:01<00:08,  1.03s/it]

 98%|█████████▊| 293/300 [05:02<00:07,  1.03s/it]

 98%|█████████▊| 294/300 [05:03<00:06,  1.03s/it]

 98%|█████████▊| 295/300 [05:04<00:05,  1.03s/it]

 99%|█████████▊| 296/300 [05:05<00:04,  1.03s/it]

 99%|█████████▉| 297/300 [05:06<00:03,  1.03s/it]

 99%|█████████▉| 298/300 [05:07<00:02,  1.03s/it]

100%|█████████▉| 299/300 [05:07<00:01,  1.03s/it]

100%|██████████| 300/300 [05:08<00:00,  1.03s/it]



,name,author,text,image
106,밀레니엄 Vol.1 - 여자를 증오한 남자들,스티그 라르손,"\n출판사 리뷰\n“기억해둬. 내가 미친년이라는 사실을.”리스베트 & 미카엘, 그 ...",http://image.bandinlunis.com/upload/product/41...
110,오페라의 유령[한글판+영문판],가스통 르루,\n출판사 리뷰\n “당신은 추악한 나를 사랑해 줄 수 있나요?”\n \n가스통 르...,http://image.bandinlunis.com/upload/product/37...
211,느빌 백작의 범죄,아멜리 노통브,\n출판사 리뷰\n노통브는 멋진 수확을 거뒀다. 심술궂은 터치가 가미된 사랑스러운 ...,http://image.bandinlunis.com/upload/product/40...
220,삼국지 5 - 개정,나관중,"\n출판사 리뷰\n작가 이문열은 <사람의 아들>, <황제를 위하여> 등 문학적 향기...",http://image.bandinlunis.com/upload/product/la...
75,신 3,베르나르베르베르,\n출판사 리뷰\n출간 즉시 프랑스 베스트셀러 1위!\r\n지난 2008년 11월 ...,http://image.bandinlunis.com/upload/product/29...
76,독방 2,카린 지에벨,\n출판사 리뷰\n프랑스 심리스릴러의 아이콘 카린 지에벨 장편소설! 코냑추리소설대상...,http://image.bandinlunis.com/upload/product/41...
219,디너 Dinner - 매드 픽션 클럽,헤르만 코흐,\n출판사 리뷰\n“밤을 지새우게 하는 놀라운 소설!”네덜란드 국민작가 헤르만 코흐...,http://image.bandinlunis.com/upload/product/34...
80,제3인류 3,베르나르 베르베르,"\n출판사 리뷰\n베르나르 베르베르 2년 만의 신작 장편! 신화와 과학, 상상력으로...",http://image.bandinlunis.com/upload/product/37...
185,사장을 죽이고 싶나,원샨,"\n이 책은\n“사장을 죽이기에 앞서, 당신은 먼저 ‘시체를 잘 숨기는’ 방법을 배...",http://image.bandinlunis.com/upload/product/41...
237,크로우 걸 2,에리크 악슬 순드,"\n출판사 리뷰\n스톡홀름에서 벌어지는 끔찍한 소년 살해 사건,그리고 부모에 의해 ...",http://image.bandinlunis.com/upload/product/39...


In [80]:
print(len(ko_data))
ko_data = ko_data[ko_data.text.isnull() == False]
ko_data.reset_index(drop = True)
print(len(ko_data))
ko_data.sample(10)

300
300


,name,author,text,image
297,별을 스치는 바람 2,이정명,\n출판사 리뷰\n이탈리아 문학상 '프레미오 셀레지오네 반카렐라' 수상작“문학은 과...,http://image.bandinlunis.com/upload/product/34...
23,고래,천명관,\n이 책은\n\t\t\t\t\t\t\t\t\t\t\t\t\t소설에 대해 우리가 가...,http://image.bandinlunis.com/upload/product/37...
265,맨발로 글목을 돌다-제35회 이상문학상 작품집(2011),"공지영, 김경욱, 전성태, 정지아, 김숨, 김태용, 김언수, 황정은",\n출판사 리뷰\n2011년도 제35회 이상문학상 작품집 출간!!\n한국문학을 사랑...,http://image.bandinlunis.com/upload/product/32...
126,소년을 위로해줘,은희경,"\n출판사 리뷰\n너, 지금 듣고 있어?\r\n“지금 바라보는 어디든 니 얼굴뿐이라...",http://image.bandinlunis.com/upload/product/32...
128,새의 선물-2판,은희경,"\n출판사 리뷰\n▶대형신인의 산실, 문학동네소설상 제1회 수상작_하나의 장르가 된...",http://image.bandinlunis.com/upload/product/32...
144,나의 작은 인형 상자(DVD포함),정유미,"\n출판사 리뷰\n세계 최고 권위를 자랑하는 볼로냐 라가치상 정유미 작가, <먼지아...",http://image.bandinlunis.com/upload/product/38...
162,촐라체,박범신,\n이 책은\n네이버 연재 기간 누적 방문자수 100만 명을 돌파한 화제의 소설. ...,http://image.bandinlunis.com/upload/product/la...
173,폭로,정형근,\n출판사 리뷰\n\t\t\t\t\t\t\t\t\t\t\t\t\t북핵 문제로 급랭된...,http://image.bandinlunis.com/upload/product/39...
291,몽유도원 1 - 가즈오의 나라 개정판,김진명,"\n출판사 리뷰\n2010년, 김진명 소설이 다시 태어났다! \r\n한국 출판역사상...",http://image.bandinlunis.com/upload/product/30...
235,모르는 사람들,이승우,\n출판사 리뷰\n\t\t\t\t\t\t\t\t\t\t\t\t\t“그러니까 세상을 ...,http://image.bandinlunis.com/upload/product/41...


In [82]:
# 1차 전처리 : 텍스트의 숫자 및 공백 제거

for i in range(len((tqdm(ko_data['text'])))) :

    text = re.sub(r'[^a-zA-Z가-힣]', " ", ko_data.iloc[i].text)
    text = re.sub(r'[ ]{2,}', " ", text)
    ko_data.set_value(i,'text', text) 

ko_data.head()



  0%|          | 0/300 [00:00<?, ?it/s]

/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,name,author,text,image
0,82년생 김지영,조남주,출판사 리뷰 사람들이 나보고 맘충이래 한국에서 여자로 살아가는 일그 공포 피로 당...,http://image.bandinlunis.com/upload/product/40...
1,7년의 밤,정유정,출판사 리뷰 새로운 상상력 역동적 서사 강렬한 메시지 한국문단의 아마존 세계문학상...,http://image.bandinlunis.com/upload/product/39...
2,두근두근 내 인생[오디오북 특별판],김애란,출판사 리뷰 차세대 한국문단의 희망 김애란 첫 장편 년 약관의 나이로 등단한 이래...,http://image.bandinlunis.com/upload/product/36...
3,채식주의자,한강,이 책은 이상문학상 수상작 몽고반점 수록 표제작인 채식주의자 년 이상문학상 수상작...,http://image.bandinlunis.com/upload/product/34...
4,바깥은 여름,김애란,출판사 리뷰 김애란 년 만의 신작 소설집제 회 이상문학상 수상작 침묵의 미래 제 ...,http://image.bandinlunis.com/upload/product/40...


In [84]:
# 1차 전처리 : 텍스트의 숫자 및 공백 제거

for i in range(len((tqdm(jp_data['text'])))) :

    text = re.sub(r'[^a-zA-Z가-힣]', " ", jp_data.iloc[i].text)
    text = re.sub(r'[ ]{2,}', " ", text)
    jp_data.set_value(i,'text', text) 

jp_data.head()



  0%|          | 0/298 [00:00<?, ?it/s]

/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,name,author,text,image
0,돌이킬 수 없는 약속,야쿠마루 가쿠,이 책은 일본 아마존 베스트셀러 년 전 버려버린 과거에서 도착한 한 통의 편지가 ...,http://image.bandinlunis.com/upload/product/40...
1,연애의 행방,히가시노 게이고,출판사 리뷰 히가시노 게이고가 연애 라는 미스터리에 도전한다 연애의 행방 은 한국...,http://image.bandinlunis.com/upload/product/41...
2,앨리스 죽이기,고바야시 야스미,출판사 리뷰 험프티 덤프티 그리핀 흰토끼가 의문의 죽음을 맞았다 살해 용의자는 앨...,http://image.bandinlunis.com/upload/product/39...
3,노르웨이의 숲,무라카미 하루키,출판사 리뷰 나를 언제까지나 잊지 마 내가 여기 있었다는 걸 기억해 줘 하루키 월...,http://image.bandinlunis.com/upload/product/40...
4,죄의 목소리,시오타 타케시,출판사 리뷰 일본 쇼와시대 최대의 미제 구리코 모리나가 사건 그 오랜 미스터리의 ...,http://image.bandinlunis.com/upload/product/41...


In [85]:
# 1차 전처리 : 텍스트의 숫자 및 공백 제거

for i in range(len((tqdm(en_data['text'])))) :

    text = re.sub(r'[^a-zA-Z가-힣]', " ", en_data.iloc[i].text)
    text = re.sub(r'[ ]{2,}', " ", text)
    en_data.set_value(i,'text', text) 

en_data.head()



  0%|          | 0/298 [00:00<?, ?it/s]

/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,name,author,text,image
0,빅 픽처,더글라스 케네디,출판사 리뷰 책장을 넘길수록 마지막 페이지가 다가오는 게 두렵다 전 세계 여 개국...,http://image.bandinlunis.com/upload/product/31...
1,"그해, 여름 손님(양장)",안드레 애치먼,출판사 리뷰 th 람다 문학상 게이 소설 부문 수장작영화 CALL ME BY YO...,http://image.bandinlunis.com/upload/product/41...
2,봉제인형 살인사건,다니엘 콜,이 책은 영국 아마존 베스트셀러 년 런던 도서전 최고의 화제작 영국 미국 일본 독...,http://image.bandinlunis.com/upload/product/41...
3,연을 쫓는 아이 - 개정,할레드호세이니,이 책은 잃어버린 순수를 회복하는 과정을 고통스러울 정도로 정직하게 그려낸 소설이...,http://image.bandinlunis.com/upload/product/la...
4,"그해, 여름 손님",안드레 애치먼,이 책은 하나의 책 세 가지 디자인으로 만나는 소설 그해 여름 손님 책표지는 띠지...,http://image.bandinlunis.com/upload/product/40...


In [86]:
# 1차 전처리 : 텍스트의 숫자 및 공백 제거

for i in range(len((tqdm(etc_data['text'])))) :

    text = re.sub(r'[^a-zA-Z가-힣]', " ", etc_data.iloc[i].text)
    text = re.sub(r'[ ]{2,}', " ", text)
    etc_data.set_value(i,'text', text) 

etc_data.head()



  0%|          | 0/300 [00:00<?, ?it/s]

/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,name,author,text,image
0,고양이 1,베르나르 베르베르,출판사 리뷰 베르베르가 이번에는 고양이의 눈으로 인간의 미래를 바라본다한국인이 가...,http://image.bandinlunis.com/upload/product/41...
1,고양이 2,베르나르 베르베르,출판사 리뷰 베르베르가 이번에는 고양이의 눈으로 인간의 미래를 바라본다 한국인이 ...,http://image.bandinlunis.com/upload/product/41...
2,파리의 아파트,기욤 뮈소,출판사 리뷰 사랑과 감동의 마에스트로 기욤 뮈소의 중독적인 스릴러 시들해져가는 삶...,http://image.bandinlunis.com/upload/product/41...
3,연금술사,파울로 코엘료,이 책은 자아의 신화 를 찾아가는 영혼의 연금술 평범한 양치기 청년 산티아고는 마...,http://image.bandinlunis.com/upload/product/27...
4,꾸뻬 씨의 행복여행,"프랑수아 를로르, 발레리 해밀",출판사 리뷰 전 유럽인의 마음을 사로잡은 한 정신과 의사의 특별한 행복론 년 프랑...,http://image.bandinlunis.com/upload/product/30...


In [99]:
!ls -al

total 590696
drwxr-xr-x  27 JEONG  staff       864  8 22 01:10 .
drwxr-xr-x  17 JEONG  staff       544  8 14 15:28 ..
-rw-r--r--@  1 JEONG  staff     10244  8 21 19:03 .DS_Store
drwxr-xr-x  15 JEONG  staff       480  8 18 23:30 .git
drwxr-xr-x  10 JEONG  staff       320  8 22 01:04 .ipynb_checkpoints
drwxr-xr-x@ 10 JEONG  staff       320  8 21 20:57 ExhbnRec
-rw-r--r--@  1 JEONG  staff      4119  7 19 23:57 README.md
-rw-r--r--   1 JEONG  staff     45945  8 21 15:16 Western_최종.ipynb
-rw-r--r--   1 JEONG  staff     14473  8 17 01:31 art2vec.ipynb
-rw-r--r--   1 JEONG  staff    504837  8 17 01:22 art_korean_750.csv
-rw-r--r--   1 JEONG  staff    299492  8 17 01:22 art_oriental_750.csv
-rw-r--r--   1 JEONG  staff   1098969  8 17 01:22 art_western_750.csv
-rw-r--r--   1 JEONG  staff    537586  8 18 23:27 arthub & 교보_doc2vec _ Simon.ipynb
-rw-r--r--   1 JEONG  staff    934264  8 18 23:30 arthub_doc2vec_이경택1.ipynb
-rw-r--r--   1 JEONG  staff  49268392  8 18 23:26 arthub_model.model
-rw-r--r-

In [97]:
all_data = pd.concat([ko_data, jp_data, en_data, etc_data])

In [103]:
all_data.to_csv('./books_all.csv')
ko_data.to_csv('./books_ko.csv')
jp_data.to_csv('./books_jp.csv')
en_data.to_csv('./books_en.csv')
etc_data.to_csv('./books_etc.csv')

In [104]:
!ls -al

total 620120
drwxr-xr-x  32 JEONG  staff      1024  8 22 01:10 .
drwxr-xr-x  17 JEONG  staff       544  8 14 15:28 ..
-rw-r--r--@  1 JEONG  staff     10244  8 21 19:03 .DS_Store
drwxr-xr-x  15 JEONG  staff       480  8 18 23:30 .git
drwxr-xr-x  10 JEONG  staff       320  8 22 01:04 .ipynb_checkpoints
drwxr-xr-x@ 10 JEONG  staff       320  8 21 20:57 ExhbnRec
-rw-r--r--@  1 JEONG  staff      4119  7 19 23:57 README.md
-rw-r--r--   1 JEONG  staff     45945  8 21 15:16 Western_최종.ipynb
-rw-r--r--   1 JEONG  staff     14473  8 17 01:31 art2vec.ipynb
-rw-r--r--   1 JEONG  staff    504837  8 17 01:22 art_korean_750.csv
-rw-r--r--   1 JEONG  staff    299492  8 17 01:22 art_oriental_750.csv
-rw-r--r--   1 JEONG  staff   1098969  8 17 01:22 art_western_750.csv
-rw-r--r--   1 JEONG  staff    537586  8 18 23:27 arthub & 교보_doc2vec _ Simon.ipynb
-rw-r--r--   1 JEONG  staff    934264  8 18 23:30 arthub_doc2vec_이경택1.ipynb
-rw-r--r--   1 JEONG  staff  49268392  8 18 23:26 arthub_model.model
-rw-r--r-